# SparkSQL
- 학습 데이터의 평균/표준편차를 구해서 저장하기
- http://spark.apache.org/docs/2.3.0/api/python/pyspark.sql.html

In [1]:
from pyspark import SparkContext, SparkConf, storagelevel

from pyspark.sql import SparkSession
import pyspark.sql.functions as psf
from pyspark.sql.types import StructType, FloatType

sc

<SparkContext master=local[*] appName=PySparkShell>

In [2]:
spark = SparkSession \
       .builder \
       .appName("sparkSQL") \
       .config("spark.sql.crossJoin.enabled", True) \
       .getOrCreate()

In [72]:
# hdfs

confidence = spark.read.text("/hyunwoo/train_data/confidence.txt")
count = spark.read.text("/hyunwoo/train_data/count.txt")
rectangle_coord = spark.read.text("/hyunwoo/train_data/rectangle_coord.txt")
token = spark.read.text("/hyunwoo/train_data/token.txt")

In [33]:
confidence.show(5)
count.show(5)
rectangle_coord.show(5)
token.show()

+------------------+
|             value|
+------------------+
| 33.91636465503324|
|36.265998570441845|
| 39.05037907624643|
|  38.5720515191172|
|  38.0590792264943|
+------------------+
only showing top 5 rows

+------------+
|       value|
+------------+
|200, 130, 60|
|200, 160, 80|
|200, 155, 72|
|200, 163, 75|
+------------+

+----------------+
|           value|
+----------------+
|197, 125, 71, 71|
|197, 123, 73, 73|
|197, 125, 71, 71|
|197, 125, 72, 72|
|196, 125, 72, 72|
+----------------+
only showing top 5 rows

+----------+
|     value|
+----------+
|0123456789|
+----------+



## describe 이용
- 하나의 token에 대해서만 데이터 확인할 경우

In [34]:
d_confidence = confidence.selectExpr("CAST(value AS Float)").describe()
d_confidence.show()

+-------+-----------------+
|summary|            value|
+-------+-----------------+
|  count|              100|
|   mean|38.34141864776611|
| stddev|4.604178031144208|
|    min|        32.896786|
|    max|        56.184433|
+-------+-----------------+



In [35]:
d_rectangle_coord = rectangle_coord.select(psf.split(psf.col("value"), ", ").alias("value")) \
                                   .select(psf.col("value").getItem(0).alias("x"),
                                           psf.col("value").getItem(1).alias("y"),
                                           psf.col("value").getItem(2).alias("w"),
                                           psf.col("value").getItem(3).alias("h")) \
                                   .selectExpr("CAST(x AS Float)", "CAST(y AS Float)", "CAST(w AS Float)", "CAST(h AS Float)") \
                                   .describe()
                                    
d_rectangle_coord.show()                

+-------+------------------+------------------+------------------+------------------+
|summary|                 x|                 y|                 w|                 h|
+-------+------------------+------------------+------------------+------------------+
|  count|               200|               200|               200|               200|
|   mean|            196.86|            124.42|            72.215|            72.215|
| stddev|1.0225596025188444|0.7852919311220703|1.3557156132908503|1.3557156132908503|
|    min|             194.0|             122.0|              69.0|              69.0|
|    max|             200.0|             126.0|              77.0|              77.0|
+-------+------------------+------------------+------------------+------------------+



In [56]:
d_count = count.select(psf.split(psf.col("value"), ", ").alias("value")) \
               .select(psf.col("value").getItem(0).alias("img"),
                       psf.col("value").getItem(1).alias("face"),
                       psf.col("value").getItem(2).alias("eye")) \
               .selectExpr("CAST(img AS Float)", "CAST(face AS Float)", "CAST(eye AS Float)") \
               .select((psf.col("face")/psf.col("img")).alias("face_ratio"),
                       (psf.col("eye")/psf.col("img")).alias("eye_ratio")) \
               .describe()
        
d_count.show()

+-------+-------------------+-------------------+
|summary|         face_ratio|          eye_ratio|
+-------+-------------------+-------------------+
|  count|                  4|                  4|
|   mean|               0.76|            0.35875|
| stddev|0.07516648189186452|0.04250000000000002|
|    min|               0.65|                0.3|
|    max|              0.815|                0.4|
+-------+-------------------+-------------------+



In [57]:
v_confidence = d_confidence.filter("summary == 'mean' or summary == 'stddev'")
v_confidence.show()

+-------+-----------------+
|summary|            value|
+-------+-----------------+
|   mean|38.34141864776611|
| stddev|4.604178031144208|
+-------+-----------------+



In [58]:
v_rectangle_coord = d_rectangle_coord.filter("summary == 'mean' or summary == 'stddev'")
v_rectangle_coord.show()

+-------+------------------+------------------+------------------+------------------+
|summary|                 x|                 y|                 w|                 h|
+-------+------------------+------------------+------------------+------------------+
|   mean|            196.86|            124.42|            72.215|            72.215|
| stddev|1.0225596025188444|0.7852919311220703|1.3557156132908503|1.3557156132908503|
+-------+------------------+------------------+------------------+------------------+



In [59]:
v_count = d_count.filter("summary == 'mean' or summary == 'stddev'")
v_count.show()

+-------+-------------------+-------------------+
|summary|         face_ratio|          eye_ratio|
+-------+-------------------+-------------------+
|   mean|               0.76|            0.35875|
| stddev|0.07516648189186452|0.04250000000000002|
+-------+-------------------+-------------------+



In [60]:
v_confidence_mean = v_confidence.filter("summary == 'mean'") \
                                .select(psf.col("value").alias("confidencee_mean")) 
                    
v_confidence_std = v_confidence.filter("summary == 'stddev'")\
                               .select(psf.col("value").alias("confidencee_std"))

v_rectangle_coord_mean = v_rectangle_coord.filter("summary == 'mean'")\
                                           .select(psf.col("x").alias("x_mean"),
                                                   psf.col("y").alias("y_mean"),
                                                   psf.col("w").alias("w_mean"),
                                                   psf.col("h").alias("h_mean"))
    
v_rectangle_coord_std = v_rectangle_coord.filter("summary == 'stddev'")\
                                         .select(psf.col("x").alias("x_std"),
                                                 psf.col("y").alias("y_std"),
                                                 psf.col("w").alias("w_std"),
                                                 psf.col("h").alias("h_std"))
    
v_count_mean = v_count.filter("summary == 'mean'") \
                      .select(psf.col("face_ratio").alias("face_ratio_mean"), 
                              psf.col("eye_ratio").alias("eye_ratio_mean")) 
                    
v_count_std = v_count.filter("summary == 'stddev'")\
                     .select(psf.col("face_ratio").alias("face_ratio_std"), 
                             psf.col("eye_ratio").alias("eye_ratio_std")) 

In [62]:
result = token.join(v_confidence_mean).join(v_confidence_std).join(v_rectangle_coord_mean).join(v_rectangle_coord_std).join(v_count_mean).join(v_count_std)
result.show()

+----------+-----------------+-----------------+------+------+------+------+------------------+------------------+------------------+------------------+---------------+--------------+-------------------+-------------------+
|     value| confidencee_mean|  confidencee_std|x_mean|y_mean|w_mean|h_mean|             x_std|             y_std|             w_std|             h_std|face_ratio_mean|eye_ratio_mean|     face_ratio_std|      eye_ratio_std|
+----------+-----------------+-----------------+------+------+------+------+------------------+------------------+------------------+------------------+---------------+--------------+-------------------+-------------------+
|0123456789|38.34141864776611|4.604178031144208|196.86|124.42|72.215|72.215|1.0225596025188444|0.7852919311220703|1.3557156132908503|1.3557156132908503|           0.76|       0.35875|0.07516648189186452|0.04250000000000002|
+----------+-----------------+-----------------+------+------+------+------+------------------+---------

## aggregation 이용
- 여러 개의 token이 있을 경우
- 실제로는 token이 각 데이터와 함께 들어올 것

In [73]:
t_token = token.select(psf.col("value").alias("token"))
t_count = t_token.join(count)
t_conf = t_token.join(confidence)
t_rec = t_token.join(rectangle_coord)

t_count.show(5)
t_conf.show(5)
t_rec.show(5)

+----------+------------+
|     token|       value|
+----------+------------+
|0123456789|200, 130, 60|
|0123456789|200, 160, 80|
|0123456789|200, 155, 72|
|0123456789|200, 163, 75|
+----------+------------+

+----------+------------------+
|     token|             value|
+----------+------------------+
|0123456789| 33.91636465503324|
|0123456789|36.265998570441845|
|0123456789| 39.05037907624643|
|0123456789|  38.5720515191172|
|0123456789|  38.0590792264943|
+----------+------------------+
only showing top 5 rows

+----------+----------------+
|     token|           value|
+----------+----------------+
|0123456789|197, 125, 71, 71|
|0123456789|197, 123, 73, 73|
|0123456789|197, 125, 71, 71|
|0123456789|197, 125, 72, 72|
|0123456789|196, 125, 72, 72|
+----------+----------------+
only showing top 5 rows



In [74]:
c_count = t_count.select(psf.col("token"), psf.split(psf.col("value"), ", ").alias("value")) \
                 .select(psf.col("token"),
                         psf.col("value").getItem(0).alias("img"),
                         psf.col("value").getItem(1).alias("face"),
                         psf.col("value").getItem(2).alias("eye")) \
                 .selectExpr("token", "CAST(img AS Float)", "CAST(face AS Float)", "CAST(eye AS Float)")
        
c_conf = t_conf.selectExpr("token", "CAST(value AS Float)")

c_rec = t_rec.select(psf.col("token"), psf.split(psf.col("value"), ", ").alias("value")) \
             .select(psf.col("token"),
                     psf.col("value").getItem(0).alias("x"),
                     psf.col("value").getItem(1).alias("y"),
                     psf.col("value").getItem(2).alias("w"),
                     psf.col("value").getItem(3).alias("h")) \
              .selectExpr("token", "CAST(x AS Float)", "CAST(y AS Float)", "CAST(w AS Float)", "CAST(h AS Float)")

In [75]:
c_count.show(5)
c_conf.show(5)
c_rec.show(5)

+----------+-----+-----+----+
|     token|  img| face| eye|
+----------+-----+-----+----+
|0123456789|200.0|130.0|60.0|
|0123456789|200.0|160.0|80.0|
|0123456789|200.0|155.0|72.0|
|0123456789|200.0|163.0|75.0|
+----------+-----+-----+----+

+----------+---------+
|     token|    value|
+----------+---------+
|0123456789|33.916363|
|0123456789|   36.266|
|0123456789| 39.05038|
|0123456789|38.572052|
|0123456789| 38.05908|
+----------+---------+
only showing top 5 rows

+----------+-----+-----+----+----+
|     token|    x|    y|   w|   h|
+----------+-----+-----+----+----+
|0123456789|197.0|125.0|71.0|71.0|
|0123456789|197.0|123.0|73.0|73.0|
|0123456789|197.0|125.0|71.0|71.0|
|0123456789|197.0|125.0|72.0|72.0|
|0123456789|196.0|125.0|72.0|72.0|
+----------+-----+-----+----+----+
only showing top 5 rows



In [86]:
def dist(x, y, w, h) :
    center_x = x + w/2
    center_y = y + h/2
    
    return ((center_x-240)**2 + (center_y-180)**2)**0.5

dist_udf = psf.udf(lambda x,y,w,h: dist(x,y,w,h), FloatType())

a_count = c_count.select(psf.col("token"),
                         (psf.col("face")/psf.col("img")).alias("face_ratio"),
                         (psf.col("eye")/psf.col("img")).alias("eye_ratio")) \
                 .groupBy("token") \
                 .agg(psf.mean("face_ratio").alias("face_ratio_mean"), psf.stddev("face_ratio").alias("face_ratio_std"),
                      psf.mean("eye_ratio").alias("eye_ratio_mean"), psf.stddev("eye_ratio").alias("eye_ratio_std"))

a_conf = c_conf.groupBy("token") \
               .agg(psf.mean("value").alias("confidence_mean"), psf.stddev("value").alias("confidence_std"))
    
a_rec = c_rec.withColumn("dist", dist_udf("x", "y", "w", "h")) \
             .withColumn("area", psf.col("w")*psf.col("h")) \
             .groupBy("token") \
             .agg(psf.mean("x").alias("x_mean"), psf.stddev("x").alias("x_std"),
                  psf.mean("y").alias("y_mean"), psf.stddev("y").alias("y_std"),
                  psf.mean("w").alias("wh_mean"), psf.stddev("w").alias("wh_std"),
                  psf.mean("dist").alias("dist_mean"), psf.stddev("dist").alias("dist_std"),
                  psf.mean("area").alias("area_mean"), psf.stddev("area").alias("area_std"))

In [87]:
a_count.show()
a_conf.show()
a_rec.show()

+----------+---------------+-------------------+--------------+-------------------+
|     token|face_ratio_mean|     face_ratio_std|eye_ratio_mean|      eye_ratio_std|
+----------+---------------+-------------------+--------------+-------------------+
|0123456789|           0.76|0.07516648189186452|       0.35875|0.04250000000000002|
+----------+---------------+-------------------+--------------+-------------------+

+----------+-----------------+-----------------+
|     token|  confidence_mean|   confidence_std|
+----------+-----------------+-----------------+
|0123456789|38.34141864776611|4.604178031144208|
+----------+-----------------+-----------------+

+----------+------+------------------+------+------------------+-------+------------------+------------------+-----------------+---------+------------------+
|     token|x_mean|             x_std|y_mean|             y_std|wh_mean|            wh_std|         dist_mean|         dist_std|area_mean|          area_std|
+----------+-----

In [88]:
result = a_count.join(a_conf, ["token"]).join(a_rec, ["token"])
result.show()

+----------+---------------+-------------------+--------------+-------------------+-----------------+-----------------+------+------------------+------+------------------+-------+------------------+------------------+-----------------+---------+------------------+
|     token|face_ratio_mean|     face_ratio_std|eye_ratio_mean|      eye_ratio_std|  confidence_mean|   confidence_std|x_mean|             x_std|y_mean|             y_std|wh_mean|            wh_std|         dist_mean|         dist_std|area_mean|          area_std|
+----------+---------------+-------------------+--------------+-------------------+-----------------+-----------------+------+------------------+------+------------------+-------+------------------+------------------+-----------------+---------+------------------+
|0123456789|           0.76|0.07516648189186452|       0.35875|0.04250000000000002|38.34141864776611|4.604178031144208|196.86|1.0225596025188444|124.42|0.7852919311220703| 72.215|1.3557156132908503|20.7131

In [89]:
# hdfs
result.write.option("header", "true").csv("/hyunwoo/train_result/result.csv")

In [90]:
reload = spark.read.option("header", "true").csv("/hyunwoo/train_result/result.csv")
reload.show()

+----------+---------------+-------------------+--------------+-------------------+-----------------+-----------------+------+------------------+------+------------------+-------+------------------+------------------+-----------------+---------+------------------+
|     token|face_ratio_mean|     face_ratio_std|eye_ratio_mean|      eye_ratio_std|  confidence_mean|   confidence_std|x_mean|             x_std|y_mean|             y_std|wh_mean|            wh_std|         dist_mean|         dist_std|area_mean|          area_std|
+----------+---------------+-------------------+--------------+-------------------+-----------------+-----------------+------+------------------+------+------------------+-------+------------------+------------------+-----------------+---------+------------------+
|0123456789|           0.76|0.07516648189186452|       0.35875|0.04250000000000002|38.34141864776611|4.604178031144208|196.86|1.0225596025188444|124.42|0.7852919311220703| 72.215|1.3557156132908503|20.7131